In [21]:
import os
from osgeo import gdal
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import box

In [22]:
def reproject_raster(input_tiff, output_tiff, target_crs):
    # Use gdal.Warp for easier reprojection
    gdal.Warp(output_tiff, input_tiff, dstSRS=f'EPSG:{target_crs}')
    print(f"Reprojected {input_tiff} to {output_tiff}")

In [23]:
def clip_raster_with_polygon(raster_path, shapefile_path, output_path, buffer_percent=1):
    # Load the shapefile
    shapes = gpd.read_file(shapefile_path)
    
    # Load the raster
    with rasterio.open(raster_path) as src:
        # Ensure shapefile CRS matches raster CRS
        if shapes.crs != src.crs:
            shapes = shapes.to_crs(src.crs)
        
        # Calculate the buffer distance as 3% of the diagonal of the bounding box
        bounds = shapes.total_bounds  # [minx, miny, maxx, maxy]
        width = bounds[2] - bounds[0]
        height = bounds[3] - bounds[1]
        diagonal = (width**2 + height**2) ** 0.5
        buffer_distance = (buffer_percent / 100) * diagonal

        # Apply the buffer to expand the clipping geometry
        buffered_shapes = shapes.copy()
        buffered_shapes['geometry'] = shapes.geometry.buffer(buffer_distance)

        # Clip the raster with the buffered geometry
        out_image, out_transform = mask(src, buffered_shapes.geometry, crop=True)
        out_meta = src.meta.copy()

        # Update metadata
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Write the clipped raster
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)
        print(f"Clipped {raster_path} to {output_path} with a {buffer_percent}% buffer")

In [24]:
def zip_output_folder(output_folder, zip_directory):
    """
    Creates a ZIP file containing all files in the output folder.
    """
    zip_filepath = os.path.join(Anaconda/Temperature/Output, "Zipped Tiffs.zip")
    with zipfile.ZipFile(zip_filepath, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                # Skip the ZIP file itself if it exists in the same directory
                if file == "Zipped Tiffs.zip":
                    continue
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=output_folder)
                zipf.write(file_path, arcname)
    print(f"Zipped all outputs to {zip_filepath}")

In [25]:
def process_rasters_in_folder(folder_path, target_crs, shapefile_path, output_folder):
    
# Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

# Loop through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            input_tiff = os.path.join(folder_path, filename)
            
            # Step 1: Reproject the raster
            reprojected_tiff = os.path.join(output_folder, f"reprojected_{filename}")
            reproject_raster(input_tiff, reprojected_tiff, target_crs)
            
            # Step 2: Clip the raster with the polygon
            clipped_tiff = os.path.join(output_folder, f"clipped_{filename}")
            clip_raster_with_polygon(reprojected_tiff, shapefile_path, clipped_tiff)

# Example usage
folder_path = "Data"
target_crs = 32609 # e.g., "4326" for WGS84 IMPORTANT NOTE JUNEAU IS ON REGION 8N which is 32608
shapefile_path = "Shapes/Upslope Area_3.shp"
output_folder = "Output"

process_rasters_in_folder(folder_path, target_crs, shapefile_path, output_folder)

Reprojected Data/tas_mean_C_cru_ts405_historical_11_2018.tif to Output/reprojected_tas_mean_C_cru_ts405_historical_11_2018.tif
Clipped Output/reprojected_tas_mean_C_cru_ts405_historical_11_2018.tif to Output/clipped_tas_mean_C_cru_ts405_historical_11_2018.tif with a 1% buffer
Reprojected Data/tas_mean_C_cru_ts405_historical_10_2018.tif to Output/reprojected_tas_mean_C_cru_ts405_historical_10_2018.tif
Clipped Output/reprojected_tas_mean_C_cru_ts405_historical_10_2018.tif to Output/clipped_tas_mean_C_cru_ts405_historical_10_2018.tif with a 1% buffer
Reprojected Data/tas_mean_C_cru_ts405_historical_09_2003.tif to Output/reprojected_tas_mean_C_cru_ts405_historical_09_2003.tif
Clipped Output/reprojected_tas_mean_C_cru_ts405_historical_09_2003.tif to Output/clipped_tas_mean_C_cru_ts405_historical_09_2003.tif with a 1% buffer
Reprojected Data/tas_mean_C_cru_ts405_historical_08_2003.tif to Output/reprojected_tas_mean_C_cru_ts405_historical_08_2003.tif
Clipped Output/reprojected_tas_mean_C_cru_